In [3]:
import numpy as np

import pandas as pd
import time
import matplotlib.pyplot as plt
import re, string, timeit
from nltk.tag.stanford import StanfordNERTagger
import pickle
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
pd.set_option('mode.chained_assignment',None)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import LinearSVC


In [5]:
import tensorflow as tf
import transformers #huggingface transformers library


In [6]:
df = pd.read_pickle('data/final_Processed_df.pkl')

In [7]:
df.head()

,authors,date,headline,link,short_description,NERtext,text,category
0,Melissa Jeltsen,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,There Were CARDINAL Mass Shootings In GPE DATE...,There Were 2 Mass Shootings In Texas Last Week...,CRIME
1,Andy McDonald,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,Will Smith Joins Diplo And PERSON For WORK_OF_...,Will Smith Joins Diplo And Nicky Jam For The 2...,ENTERTAINMENT
2,Ron Dicker,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,PERSON For The ORDINAL Time At DATE The actor ...,Hugh Grant Marries For The First Time At Age 5...,ENTERTAINMENT
3,Ron Dicker,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,PERSON Blasts GPE PERSON And NORP In New Artwo...,Jim Carrey Blasts Castrato Adam Schiff And Dem...,ENTERTAINMENT
4,Ron Dicker,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",PERSON Uses PERSON Poop Bags To Pick Up After ...,Julianna Margulies Uses Donald Trump Poop Bags...,ENTERTAINMENT


In [8]:
for i in df['text'].head():
    print(i)

There Were 2 Mass Shootings In Texas Last Week But Only 1 On TV She left her husband He killed their children Just another day in America
Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup 's Official Song Of course it has a song
Hugh Grant Marries For The First Time At Age 57 The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony
Jim Carrey Blasts Castrato Adam Schiff And Democrats In New Artwork The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump
Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog The Dietland actress said using the bags is a really cathartic therapeutic moment


In [9]:

for i in df['NERtext'].head():
    print(i)

There Were CARDINAL Mass Shootings In GPE DATE But DATE On TV She left her husband He killed their children Just another day in GPE
Will Smith Joins Diplo And PERSON For WORK_OF_ART Of course it has a song
PERSON For The ORDINAL Time At DATE The actor and his longtime girlfriend PERSON tied the knot in a civil ceremony
PERSON Blasts GPE PERSON And NORP In New Artwork The actor gives NORP an ass-kicking for not fighting hard enough against PERSON
PERSON Uses PERSON Poop Bags To Pick Up After Her Dog The NORP actress said using the bags is a really cathartic therapeutic moment


In [10]:
###X_train, X_test, y_train, y_test  =train_test_split(df['text'], df['category'], test_size=0.2, random_state=1, stratify=df['category'])
#X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=1, stratify=y_train)

In [11]:
class model():
    def __init__(self, df, textColumn = 'text', addAuthor = False, SEED = 2):
        self.SEED = SEED
        self.df = df
        self.textColumn = textColumn
        self.le = LabelEncoder()
        self.df['category_code'] = self.le.fit_transform(df['category'])

        if addAuthor:
            self.df[textColumn] = self.df[textColumn] +' ' + self.df['authors']
    def lem(self, phrase, rarewords):
        stop_words_ = list(set(stopwords.words('english')))
        wn = WordNetLemmatizer()
        return " ".join([wn.lemmatize(words.lower(), pos="v") for words in word_tokenize(phrase) if (words not in (stop_words_ + rarewords))])

    def fit(self, rarewords = []):
        self.df[self.textColumn] = self.df[self.textColumn].apply(lambda x: self.lem(x, rarewords))
        print('-----------Sample Text---------')
        for i in self.df[self.textColumn].head(3):
            print(i)
        self.X_train, self.X_test, self.y_train, self.y_test  =train_test_split(self.df[self.textColumn], self.df['category_code'], test_size=0.2, random_state=1, stratify=self.df['category_code'])
        self.X_train, self.X_val, self.y_train, self.y_val  = train_test_split(self.X_train, self.y_train, test_size=0.25, random_state=1, stratify=self.y_train)
        vect_word = TfidfVectorizer(max_features=60000, lowercase=False, analyzer='word',tokenizer=word_tokenize,ngram_range=(1,3),dtype=np.float32)
        tr_vect = vect_word.fit_transform(self.X_train)
        vl_vect = vect_word.transform(self.X_val)
        #test_vect = vect_word.transform(X_test)
        
        #logistic regression
        self.lgr = LogisticRegression(solver='saga',multi_class='multinomial', max_iter=1000, C=1,verbose = True, n_jobs = -1, random_state = self.SEED).fit(tr_vect, self.y_train)
        self.lgr_y_pred = self.lgr.predict(vl_vect)
        print('LGR accuracy: ', (self.lgr_y_pred == self.y_val).sum()/len(self.y_val))

        #linear SVC
        
        self.svc = LinearSVC(verbose = True, random_state = self.SEED).fit(tr_vect, self.y_train)
        self.svc_y_pred = self.svc.predict(vl_vect)
        print('LinearSVC: ', (self.svc_y_pred == self.y_val).sum()/len(self.y_val))

In [10]:
SEED = 2
textColumnToUse = 'NERtext'
addAuthor = True
nermodel = model(df, textColumnToUse, addAuthor, SEED)
nermodel.fit()

-----------Sample Text---------
there be cardinal mass shoot in gpe date but date on tv she leave husband he kill children just another day gpe melissa jeltsen
will smith join diplo and person for work_of_art of course song andy mcdonald
person for the ordinal time at date the actor longtime girlfriend person tie knot civil ceremony ron dicker


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


convergence after 22 epochs took 34 seconds
LGR accuracy: 

[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   34.0s finished


 0.7082721366159668
[LibLinear]LinearSVC:  0.7256229618381419


In [11]:
textColumnToUse = 'text'
model = model(df, textColumnToUse, addAuthor)
model.fit()

-----------Sample Text---------
there be 2 mass shoot in texas last week but only 1 on tv she leave husband he kill children just another day america melissa jeltsen
will smith join diplo and nicky jam for the 2018 world cup 's official song of course song andy mcdonald
hugh grant marry for the first time at age 57 the actor longtime girlfriend anna eberstein tie knot civil ceremony ron dicker


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


convergence after 19 epochs took 29 seconds
LGR accuracy:  0.7315974210251176


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   29.1s finished


[LibLinear]LinearSVC:  0.7522839859600209


In [53]:
# without author NER LGR accuracy:  0.5542804510716686 LinearSVC:  0.554429812551343, BERT accuracy: 0.6484048366546631
# without author LGR accuracy:  0.6009310198899703 LinearSVC:  0.6112618555674492 BERT accuracy: 0.6991676092147827

# with Author NER LGR accuracy:  0.7082721366159668 LinearSVC:   0.7256229618381419
# with Author LGR accuracy:  0.7315974210251176 LinearSVC:  0.7522839859600209



In [12]:
df.groupby(by='category').size().sort_values(ascending = False)

category
POLITICS          32739
WELLNESS          17827
ENTERTAINMENT     16058
TRAVEL             9887
STYLE & BEAUTY     9649
PARENTING          8677
HEALTHY LIVING     6694
QUEER VOICES       6314
FOOD & DRINK       6226
BUSINESS           5937
COMEDY             5175
SPORTS             4884
BLACK VOICES       4528
HOME & LIVING      4195
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3651
WOMEN              3490
IMPACT             3459
DIVORCE            3426
CRIME              3405
MEDIA              2815
WEIRD NEWS         2670
GREEN              2622
WORLDPOST          2579
RELIGION           2556
STYLE              2254
SCIENCE            2178
WORLD NEWS         2177
TASTE              2096
TECH               2082
MONEY              1707
ARTS               1509
FIFTY              1401
GOOD NEWS          1398
ARTS & CULTURE     1339
ENVIRONMENT        1323
COLLEGE            1144
LATINO VOICES      1129
CULTURE & ARTS     1030
EDUCATION          1004
dtype: 

In [46]:
test_features = ['TASTE', 'WEDDINGS']
test_df = df[df['category'].isin(test_features) ]

In [47]:
for i in test_df.head()['text']:
    print(i)

It 's Weird That American Restaurants Serve Ice Water In Winter But why do we even have ice in our drinks in the first place
Pineapple Casserole The Southern Dish That 's A Paradox Of Flavors It 's got pineapple cheddar and a whole lot of butter
How To Actually Get A Bartender 's Attention Plus other things they wish you knew
Diet Coke 's Millennial-Inspired Makeover Leaves People Befuddled It 's not like a regular soda it 's a cool soda
We Tested The New Tearless Onions To See If They Really Work Put away your goggles people


In [48]:
for i in test_df.head()['NERtext']:
    print(i)

It 's Weird That American Restaurants Serve Ice Water In Winter But why do we even have ice in our drinks in the ORDINAL place
Pineapple Casserole The Southern Dish That 's A Paradox Of Flavors It 's got pineapple cheddar and a whole lot of butter
How To Actually Get A Bartender 's Attention Plus other things they wish you knew
PERSON Millennial - Inspired Makeover Leaves People Befuddled It 's not like a regular soda it 's a cool soda
We Tested The New Tearless Onions To See If They Really Work Put away your goggles people


In [49]:
SEED = 2
textColumnToUse = 'NERtext'
addAuthor = True
nermodel = model(test_df, textColumnToUse, addAuthor, SEED)
nermodel.fit()

-----------Sample Text---------
it 's weird that american restaurants serve ice water in winter but even ice drink ordinal place todd van luling
pineapple casserole the southern dish that 's a paradox of flavor it 's get pineapple cheddar whole lot butter kristen aiken
how to actually get a bartender 's attention plus things wish know taylor pittman


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


convergence after 20 epochs took 0 seconds
LGR accuracy:  0.9834782608695652
[LibLinear]LinearSVC:  0.9904347826086957


In [50]:
SEED = 2
textColumnToUse = 'text'
addAuthor = True
nermodel = model(test_df, textColumnToUse, addAuthor, SEED)
nermodel.fit()

-----------Sample Text---------
it 's weird that american restaurants serve ice water in winter but even ice drink first place todd van luling
pineapple casserole the southern dish that 's a paradox of flavor it 's get pineapple cheddar whole lot butter kristen aiken
how to actually get a bartender 's attention plus things wish know taylor pittman


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


convergence after 21 epochs took 0 seconds
LGR accuracy:  0.9904347826086957
[LibLinear]LinearSVC:  0.9947826086956522


In [60]:
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()
def lem(phrase):
        return " ".join([wn.lemmatize(words.lower(), pos="v") for words in word_tokenize(phrase) if (words not in stop_words_)])

In [61]:
df_lem = df.copy()

In [62]:
df_lem['text'] = df['text'].apply(lambda x: lem(x))

In [63]:
fulltext = ' '.join(df_lem['text'])

In [64]:
wordFreq = nltk.FreqDist(word_tokenize(fulltext))

In [65]:
wordFreq

FreqDist({'the': 82658, "'s": 79341, 'i': 46365, 'be': 40582, 'to': 40318, 'a': 31798, 'in': 30472, "n't": 24366, 'of': 23702, 'for': 21983, ...})

In [72]:
rareWords = len([k for k, v in sorted(wordFreq.items(), key=lambda item: item[1]) if v ==1])

In [76]:
textColumnToUse = 'text'
model = model(df, textColumnToUse, addAuthor,rareWords)
model.fit()

-----------Sample Text---------
there be 2 mass shoot in texas last week but only 1 on tv she leave husband he kill children just another day america melissa jeltsen melissa jeltsen
will smith join diplo and nicky jam for the 2018 world cup 's official song of course song andy mcdonald andy mcdonald
hugh grant marry for the first time at age 57 the actor longtime girlfriend anna eberstein tie knot civil ceremony ron dicker ron dicker


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


convergence after 21 epochs took 27 seconds
LGR accuracy:  0.7376216673719848


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   27.6s finished


[LibLinear]LinearSVC:  0.7584575937865624


In [12]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping

In [13]:
types = df.category.unique()
def get_type_index(string):
    return list(types).index(string)
df['type_index'] = df['category'].apply(get_type_index)

In [14]:
train, test = train_test_split(df)
train, val = train_test_split(train)

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
trunc_type = "post"
pad_type = "post"
oov_tok = "<OOV>"
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(df.text.values)

In [17]:
one_hot_labels = tf.keras.utils.to_categorical(train.type_index.values, num_classes=41)
val_labels= tf.keras.utils.to_categorical(val.type_index.values, num_classes=41)

In [18]:
import transformers
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-large-uncased')

In [19]:
maxlen=200
train_input_ids = [tokenizer.encode(str(i), max_length = maxlen , pad_to_max_length = True) for i in train.text.values]
val_input_ids = [tokenizer.encode(str(i), max_length = maxlen , pad_to_max_length = True) for i in val.text.values]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [24]:
def create_model(): 
    input_word_ids = tf.keras.layers.Input(shape=(maxlen,), dtype=tf.int32,
                                           name="input_word_ids")
    bert_layer = transformers.TFBertModel.from_pretrained('bert-large-uncased')
    bert_outputs = bert_layer(input_word_ids)[0]
    pred = tf.keras.layers.Dense(41, activation='softmax')(bert_outputs[:,0,:])
    
    model = tf.keras.models.Model(inputs=input_word_ids, outputs=pred)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.00001), metrics=['accuracy'])
    return model

In [28]:
tf.test.gpu_device_name()

''

In [25]:
use_tpu = False
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()
    
model.summary()

Some layers from the model checkpoint at bert-large-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 200)]             0         
_________________________________________________________________
tf_bert_model_1 (TFBertModel TFBaseModelOutputWithPool 335141888 
_________________________________________________________________
tf.__operators__.getitem_1 ( (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 41)                42025     
Total params: 335,183,913
Trainable params: 335,183,913
Non-trainable params: 0
_________________________________________________________________


In [27]:
batch_size = 64

history=model.fit(np.array(train_input_ids), one_hot_labels,validation_data = (np.array(val_input_ids), val_labels),epochs = 4, batch_size = batch_size)

Epoch 1/4


ResourceExhaustedError:  OOM when allocating tensor with shape[64,16,200,200] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node model_1/tf_bert_model_1/bert/encoder/layer_._15/attention/self/dropout_119/dropout/random_uniform/RandomUniform (defined at C:\ProgramData\Anaconda3\lib\site-packages\transformers\models\bert\modeling_tf_bert.py:272) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_74266]

Function call stack:
train_function


In [ ]:
epochs = [i for i in range(4)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
fig.set_size_inches(20,10)

ax[0].plot(epochs , train_acc , 'go-' , label = 'Training Accuracy')
ax[0].plot(epochs , val_acc , 'ro-' , label = 'Testing Accuracy')
ax[0].set_title('Training & Testing Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

ax[1].plot(epochs , train_loss , 'go-' , label = 'Training Loss')
ax[1].plot(epochs , val_loss , 'ro-' , label = 'Testing Loss')
ax[1].set_title('Training & Testing Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")
plt.show()